In [2]:
#mudei os valores de x e y e mudei  batchsize, sendo alterado para 3
import torch
from torch.utils.data import Dataset, DataLoader

# Definindo os dados de entrada (x) e as saídas (y) com novos valores
x = [[2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13]]
y = [[5], [9], [13], [17], [21], [25]]

# Convertendo para tensores e movendo para o dispositivo adequado (GPU ou CPU)
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

# Verificando se há GPU disponível e movendo os tensores para o dispositivo adequado
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

# Definindo uma classe para o Dataset
class MeuDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

# Criando o Dataset
ds = MeuDataset(x, y)

# Definindo o DataLoader para carregar os dados em mini-lotes
# Mudando o batch_size para 3 e habilitando o embaralhamento dos dados
dl = DataLoader(ds, batch_size=3, shuffle=True)

# Carregando e imprimindo os dados em mini-lotes
for x_batch, y_batch in dl:
    print(x_batch, y_batch)


tensor([[ 8.,  9.],
        [10., 11.],
        [12., 13.]]) tensor([[17.],
        [21.],
        [25.]])
tensor([[4., 5.],
        [6., 7.],
        [2., 3.]]) tensor([[ 9.],
        [13.],
        [ 5.]])


In [4]:
from torch.optim import SGD
import torch
import torch.nn as nn

# Definindo a rede neural personalizada
class MinhaRedeNeural(nn.Module):
    def __init__(self):
        # Chamamos o super().__init__() para garantir que estamos herdando corretamente
        super().__init__()
        self.layer1 = nn.Linear(2, 8)  # Primeira camada linear
        self.ativacao = nn.ReLU()      # Função de ativação ReLU
        self.layer2 = nn.Linear(8, 1)  # Segunda camada linear

    # Quando passamos algo pela rede, ela chama a função forward
    def forward(self, x):
        x = self.layer1(x)         # Passando pela primeira camada
        x = self.ativacao(x)       # Aplicando a função de ativação
        x = self.layer2(x)         # Passando pela segunda camada
        return x

# Inicializando o modelo
modelo = MinhaRedeNeural()

# Função de perda (MSELoss) para medir o erro quadrático médio
funcao_perda = nn.MSELoss()

# Otimizador SGD (Gradiente Descendente Estocástico) com taxa de aprendizado de 0.001
otimizador = SGD(modelo.parameters(), lr=0.001)

# Lista para armazenar os valores das perdas durante o treinamento
perdas = []

# Laço de treinamento (50 épocas)
for _ in range(50):  # 50 épocas
    for dados in dl:  # Iterando sobre os mini-lotes (DataLoader)
        otimizador.zero_grad()  # Zera os gradientes antes de cada iteração
        x1, y1 = dados  # Dados de entrada (x1) e saída (y1)
        
        # Calculando o valor da perda
        valor_perda = funcao_perda(modelo(x1), y1)
        
        # Calculando os gradientes da função de perda em relação aos parâmetros da rede
        valor_perda.backward()

        # Atualizando os pesos e vieses da rede neural com base nos gradientes calculados
        otimizador.step()
        
        # Armazenando o valor da perda para acompanhar o progresso do treinamento
        perdas.append(valor_perda.detach().numpy())


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Definindo os dados de entrada (x) e saída (y)
x = [[1, 2], [3, 4], [5, 6], [7, 8]]
y = [[3], [7], [11], [15]]

# Definindo o dispositivo (GPU ou CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Criando a classe MyDataSet para o DataLoader
class MeuDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

# Criando o dataset e o DataLoader
ds = MeuDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

# Modelo usando nn.Sequential
# Esse modelo é equivalente ao modelo anterior, mas definido de forma mais simples
modelo = nn.Sequential(
    nn.Linear(2, 8),  # Primeira camada linear
    nn.ReLU(),        # Função de ativação ReLU
    nn.Linear(8, 1)   # Segunda camada linear
).to(device)

# Função de perda (MSELoss)
funcao_perda = nn.MSELoss()

# Otimizador SGD
otimizador = torch.optim.SGD(modelo.parameters(), lr=0.001)

# Lista para armazenar os valores das perdas durante o treinamento
perdas = []

# Laço de treinamento (50 épocas)
for _ in range(50):  # 50 épocas
    for dados in dl:  # Iterando sobre os mini-lotes (DataLoader)
        otimizador.zero_grad()  # Zera os gradientes antes de cada iteração
        x1, y1 = dados  # Dados de entrada (x1) e saída (y1)
        
        # Calculando o valor da perda
        valor_perda = funcao_perda(modelo(x1), y1)
        
        # Calculando os gradientes da função de perda em relação aos parâmetros da rede
        valor_perda.backward()

        # Atualizando os pesos e vieses da rede neural com base nos gradientes calculados
        otimizador.step()
        
        # Armazenando o valor da perda para acompanhar o progresso do treinamento
        perdas.append(valor_perda.detach().numpy())


In [8]:
from torch.optim import SGD
import time

# treinando o modelo

loss_func = nn.MSELoss()
opt = SGD(modelo.parameters(),lr = 0.001)
loss_history = []
start = time.time()

modelo.train()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(modelo(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)

end = time.time()
print(end - start)

0.042115211486816406


In [10]:
val = [[8,9],[10,11],[1.5,2.5]]
modelo(torch.tensor(val).float().to(device))

tensor([[16.3046],
        [19.8488],
        [ 4.7860]], grad_fn=<AddmmBackward0>)